# Video Laplacian-based Edge Detection

<table style="margin-left: 0px; margin-top: 20px; margin-bottom: 20px;">
<tr>
<td style="width:120px; padding-top: 10px; padding-bottom: 10px;"><img src="assets/Logo-TSP-IPP.png" style="height: 130px;"></td>
<td style="padding-left: 12px;">
<table style="width: 100%;">
<tr>
<th style="text-align: left; width: 80px;">File</th>
<td style="text-align: left;">OpenCV_VideoLaplacianEdgeDetection.ipynb</td>
</tr>
<tr>
<th style="text-align: left;">Author</th>
<td style="text-align: left;">Nicolas ROUGON</td>
</tr>
<tr>
<th style="text-align: left;">Affiliation</th>
<td style="text-align: left;">Institut Polytechnique de Paris &nbsp;|&nbsp; Telecom SudParis &nbsp;|&nbsp; ARTEMIS Department</td>
</tr>
<tr>
<th style="text-align: left;">Date</th>
<td style="text-align: left;">July 31, 2022</td>
</tr>
<tr>
<th style="text-align: left;">Description</th>
<td style="text-align: left;">OpenCV sample routine &nbsp;>&nbsp; Laplacian-based edge detection in video stream</td>
</tr>
</table>
</td>
</tr>
</table>
    
<b>OpenCV Documentation</b>
<ul style="margin-top: 2px;">
<li><a href="https://docs.opencv.org/4.6.0/d8/dfe/classcv_1_1VideoCapture.html">The VideoCapture class</a></li>
<li><a href="https://docs.opencv.org/4.6.0/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1">Gaussian filtering</a></li>
<li><a href="https://docs.opencv.org/4.6.0/d4/d86/group__imgproc__filter.html#gad78703e4c8fe703d479c1860d76429e6">Finite Difference Laplacian operator</a></li>
</ul>

<b>Tutorial</b>
<ul style="margin-top: 1px;">
<li><a href="https://docs.opencv.org/4.6.0/d5/dc4/tutorial_video_input_psnr_ssim.html">How to read a video stream</a></li>
<li><a href="https://docs.opencv.org/4.6.0/d5/db5/tutorial_laplace_operator.html">Laplacian operator</a></li>
</ul>

In [4]:
import cv2 as cv
import numpy as np
import sys

from enum import Enum

sys.path.append("c:/Users/rougon/Notebooks")      # TO BE SET ACCORDING TO YOUR LOCAL INSTALLATION
from OpenCV_Image_Utilities import *              # Provides overlay_uchar_image()
from OpenCV_ImageProcessing_Utilities import *    # Provides DoGFilter() FindZeroCrossings()
                                                  # Provides class SecondOrderMoment
##
## Global Variables
##

class LaplacianFilter(Enum):
    LoG = 1
    DoG = 2

# - Gaussian smoothing
trackbarStdDeviation_name = "10*StdDev"
trackbarStdDeviation_max_value = 75         # Trackbar max value
trackbarStdDeviation_min_value = 1          # Trackbar min value
trackbarStdDeviation_value = 1              # Trackbar value
trackbarStdDeviation_stepsize = 0.1         # Quantization step
sigma_default_value = 1.0                   # Gaussian kernel std deviation default value

# - Laplacian filter type
trackbarLaplacianFilter_name = "LoG | DoG"
trackbarLaplacianFilter_max_value = 1       # Trackbar max value
trackbarLaplacianFilter_value = 0           # Trackbar value
LaplacianFilter_default_value = 0           # Default Laplacian filter

# - Laplacian kernel half size
trackbarKernelHalfSize_name = "Half-width"
trackbarKernelHalfSize_max_value = 3        # Trackbar max value
trackbarKernelHalfSize_min_value = 1        # Trackbar min value
trackbarKernelHalfSize_value = 3            # Trackbar value
KernelHalfSize_default_value = 3            # Default kernel half size

# - DoG filter
DoGStdDeviationRatio = 1.6                  # Ratio of Gaussian std deviations

# - Contrast estimator
trackbarContrastEstimator_name = "Grad | Var"
trackbarContrastEstimator_max_value = 1     # Trackbar max value
trackbarContrastEstimator_value = 0         # Trackbar value
contrast_estimator_default_value = 0        # Default contrast estimator

# - Contrast threshold
trackbarContrastThreshold_name = "Threshold"
trackbarContrastThreshold_max_value = 255   # Trackbar max value 
trackbarContrastThreshold_value = 128       # Trackbar value
contrast_threshold_default_value = 128.     # Contrast threshold default value

# Edge map overlay
edge_color = 255
edge_color3 = (0,0,255)

##
## IMAGE PIPELINE COMPONENTS 
##

#
# Graphical User Interface 
#
def create_GUI():
                                    # Windows
    global window_settings_name     # GUI
    global window_overlay_name      # Displays edge map onto original image
    
    window_name_prefix = 'OpenCV Demo |  Edge detection > '
    
    # Create windows
    # - for edge map overlay
    window_overlay_name = window_name_prefix + "Detected edges"
    cv.namedWindow(window_overlay_name, cv.WINDOW_AUTOSIZE)
    
    # - for hyperparameter settings
    window_settings_name = window_name_prefix + 'Settings'
    cv.namedWindow(window_settings_name, cv.WINDOW_AUTOSIZE)
    
    # Create trackbars
    # - for Gaussian kernel std deviation
    cv.createTrackbar(trackbarStdDeviation_name, window_settings_name, 0,
                      trackbarStdDeviation_max_value, process_display_callback)
    cv.setTrackbarMin(trackbarStdDeviation_name, window_settings_name, 
                      trackbarStdDeviation_min_value)
    
    # - for Laplacian filter type (LoG | DoG)
    cv.createTrackbar(trackbarLaplacianFilter_name, window_settings_name, 0,
                      trackbarLaplacianFilter_max_value,process_display_callback)
 
    # - for Laplacian kernel half-width (LoG filter)
    cv.createTrackbar(trackbarKernelHalfSize_name, window_settings_name, 0,
                      trackbarKernelHalfSize_max_value, process_display_callback)
    cv.setTrackbarMin(trackbarKernelHalfSize_name, window_settings_name,
                      trackbarKernelHalfSize_min_value)
    
    # - for contrast estimator
    cv.createTrackbar(trackbarContrastEstimator_name, window_settings_name, 0,
                      trackbarContrastEstimator_max_value, process_display_callback)
    
    # - for contrast threshold 
    cv.createTrackbar(trackbarContrastThreshold_name, window_settings_name, 0,
                      trackbarContrastThreshold_max_value, process_display_callback)

    # Set trackbar default positions
    # - for Laplacian filter type (LoG | DoG)
    trackbarLaplacianFilter_value = LaplacianFilter_default_value
    cv.setTrackbarPos(trackbarLaplacianFilter_name, window_settings_name,
                      trackbarLaplacianFilter_value)
    
    # - for Gaussian kernel std deviation
    trackbarStdDeviation_value = (int)(sigma_default_value / trackbarStdDeviation_stepsize)
    cv.setTrackbarPos(trackbarStdDeviation_name, window_settings_name,
                      trackbarStdDeviation_value)
    
    # - for Laplacian kernel half-width (LoG filter)
    trackbarKernelHalfSize_value = KernelHalfSize_default_value
    cv.setTrackbarPos(trackbarKernelHalfSize_name, window_settings_name,
                      trackbarKernelHalfSize_value)
    
    # - for contrast estimator
    trackbarContrastEstimator_value = contrast_estimator_default_value
    cv.setTrackbarPos(trackbarContrastEstimator_name, window_settings_name,
                      trackbarContrastEstimator_value)
    
    # - for contrast threshold 
    trackbarContrastThreshold_value = (int)(contrast_threshold_default_value)
    cv.setTrackbarPos(trackbarContrastThreshold_name, window_settings_name,
                      trackbarContrastThreshold_value)

#
# Video stream initialization
# 
def initialize_stream():
    global cap
    global frame, frame_overlay
    global laplacian_map, laplacian_ZC, contrast_map, high_contrast_mask
    global isGrayCamera
    
    # Open default camera
    cap = cv.VideoCapture(0)

    # Check if camera opening is successful
    if not cap.isOpened():
        sys.exit("! Cannot open default camera")

    # Get camera properties and initialize various variables
    # - Get frame dimensions
    ncols = (int)(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    nrows = (int)(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    
    # - Check for color camera
    status, frame = cap.read()       # Get frame
    if not status:
        sys.exit("! Cannot grab frame from default camera")
    if frame.ndim == 1:
        isGrayCamera = True
    else:
        isGrayCamera = False
    
    # Create contrast / Laplacian / edge maps & edge overlay images
    laplacian_map = np.empty((nrows, ncols), dtype="float32")     # Laplacian map
    laplacian_ZC = np.empty((nrows, ncols), dtype="uint8")        # Laplacian ZC
    contrast_map = np.empty((nrows, ncols), dtype="float32")      # Contrast map
    high_contrast_mask = np.empty((nrows, ncols), dtype="uint8")  # Thresholded contrast map
    frame_overlay = np.empty((nrows, ncols, 3), dtype="uint8")    # Edge map overlay
    
#
# Input & Data preparation
#
def grab_preprocess():
    global frame, frame_gray32f
    
    # Get a new frame from camera
    status, frame = cap.read()
    
    # Convert image to graylevel if appropriate
    if isGrayCamera:
        the_frame = frame
    else:
        the_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Convert to float32 for improved accuracy
    frame_gray32f = the_frame.astype(dtype="float32")

#
# Trackbar callback
#
def process_display_callback(value):
    global sigma                     # Gaussian std deviation
    global sigma2                    # 2nd Gaussian std deviation (DoG)
    global laplacian_filter_type     # Laplacian filter type (LoG | DoG)
    global laplacian_ksize           # Laplacian kernel size
    global contrast_estimator        # Contrast estimator
    global contrast_threshold_value  # Contrast threshold

    # Get trackbars positions
    trackbarLaplacianFilter_value = cv.getTrackbarPos(trackbarLaplacianFilter_name, window_settings_name)
    trackbarStdDeviation_value = cv.getTrackbarPos(trackbarStdDeviation_name, window_settings_name)
    trackbarKernelHalfSize_value = cv.getTrackbarPos(trackbarKernelHalfSize_name, window_settings_name)
    trackbarContrastEstimator_value = cv.getTrackbarPos(trackbarContrastEstimator_name, window_settings_name)
    trackbarContrastThreshold_value = cv.getTrackbarPos(trackbarContrastThreshold_name, window_settings_name)
    
    # Set hyperparameter values from trackbars
    if trackbarLaplacianFilter_value == 0:
        laplacian_filter_type = LaplacianFilter.LoG
    elif trackbarLaplacianFilter_value == 1:
        laplacian_filter_type = LaplacianFilter.DoG

    sigma = (float)(trackbarStdDeviation_value)*trackbarStdDeviation_stepsize
    sigma2 = DoGStdDeviationRatio*sigma
    laplacian_ksize = 2*trackbarKernelHalfSize_value + 1
    contrast_estimator = trackbarContrastEstimator_value
    contrast_threshold_value = (float)(trackbarContrastThreshold_value)
    
    #  Gradient-based and variance-based contrasts have different dynamics.
    #  This issue is fixed by rescaling the threshold parameter
    if contrast_estimator == 1:    # Local std deviation
        contrast_threshold_value *= 0.1
    
    # Processing & Visualization
    process_display()

#
# Processing & Visualization
#
def process_display():
    global laplacian_map, laplacian_ZC, contrast_map, high_contrast_mask
    global frame_overlay
    global contrast_threshold_value
    global laplacian_filter_type
    
    # Preprocessing > Gaussian filtering
    # - Kernel size is set to 0, and is automatically estimated from sigma
    # - Output image has the same type as the input image. 
    #   Computations can be performed in uint8 with minimum truncation error.
    #   For improved accuracy, it is better to operate on the float32 version of the input image
    frame_smoothed = cv.GaussianBlur(frame_gray32f, (0,0), sigma)
  
    # Compute regularized Laplacian map
    if laplacian_filter_type is LaplacianFilter.LoG:     # LoG filter
        # Apply FD Laplacian kernel to Gaussian smoothed input image
        laplacian_map = cv.Laplacian(frame_smoothed, cv.CV_32F, laplacian_map, laplacian_ksize)
    elif laplacian_filter_type is LaplacianFilter.DoG:   # DoG filter
        # Apply DoG kernel to raw input image
        laplacian_map, gaussian_ksize1, gaussian_ksize2 = DoGFilter(frame_gray32f, sigma, sigma2, 1, laplacian_map)
        
    # Detect Laplacian zero crossings (ZC)
    laplacian_ZC = FindZeroCrossings(laplacian_map, laplacian_ZC)
    
    # Filter Laplacian ZC according to local contrast
    # - Compute contrast map
    if contrast_estimator == 0:    # Sobel gradient norm
        contrast_map = SobelContrastMap(frame_smoothed, cv.NORM_L2, contrast_map)
    elif contrast_estimator == 1:  # Local std deviation
        contrast_map = LocalVarianceMap(frame_smoothed, 3, SecondOrderMoment.STD_DEVIATION, contrast_map)
    
    # - Threshold contrast map
    #   - Output image type is derived from input image (i.e. np.float32)
    #   - Gradient-based and variance-based contrasts have different dynamics.
    #     This issue is fixed by rescaling the threshold parameter

    if contrast_estimator == 1:    # Local std deviation
        contrast_threshold_value *= 0.1
        
    retval, high_contrast_mask = cv.threshold(contrast_map, contrast_threshold_value, 
                                              255, cv.THRESH_BINARY)
    
    #- Select high-contrast ZC
    #   > docs.opencv.org/4.7.0/d3/d63/classcv_1_1Mat.html#adf88c60c5b4980e05bb556080916978b
    #     docs.opencv.org/4.7.0/d2/de8/group__core__array.html#ga60b4d04b251ba5eb1392c34425497e14
    high_contrast_mask = high_contrast_mask.astype(dtype="uint8")
    filtered_ZC = cv.bitwise_and(high_contrast_mask, laplacian_ZC)
    
    # Alternate NumPy-based implementation
    # filtered_ZC = 255*(np.logical_and(high_contrast_mask, laplacian_ZC)).astype(dtype="uint8")
    
    # Overlay edge map on original image
    frame_overlay = overlay_uchar_image(frame, filtered_ZC, 
                                        edge_color, edge_color3, frame_overlay)
    
    # Display result
    cv.imshow(window_overlay_name, frame_overlay)

##
## MAIN ROUTINE
##

def application():
    # Open camera & Get its features
    initialize_stream()
    
    # Grab & preprocess frame
    grab_preprocess()
    
    # GUI creation
    create_GUI()

    # Invoke callback routine to initialize and process
    process_display_callback(trackbarStdDeviation_value)
    
    # Process video stream
    while True:
        # Grab & preprocess frame
        grab_preprocess()
        
        # Processing & Visualization
        process_display()
        
        # Listen to next event
        if cv.waitKey(5) >= 0:
            break

    # Release camera
    cap.release()
    
    # Destroy windows
    cv.destroyAllWindows()

<b>Run the application</b>

In [5]:
# Run application
application()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 480 but corresponding boolean dimension is 640

error: OpenCV(4.6.0) C:\b\abs_74oeeuevib\croots\recipe\opencv-suite_1664548340488\work\modules\core\src\arithm.cpp:214: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'
